## $\color{Green}{\text{GMD Table 3: GMSL from Modules}}$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{Module data from Bob's run on Amarel}}$

In [2]:
# Load FACTS workflow data as a dataframe.
df = pd.read_fwf('../../../001_IP_names_Modules_Data/mod-submod-data_global_Bob_amarel_V2.txt')
df

,Component,Module,sub_Module,Data_File
0,GSAT,fair,temperature,temperature.fair.temperature_gsat.nc
1,TE,tlm,oceandynamics,ocean.tlm.oceandynamics_globalsl.nc
2,TE,ipccar5,thermalexpansion,XXX
3,Glaciers,kopp14,glaciers,XXX
4,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_globalsl.nc
5,Glaciers,ipccar5,glacierscmip6,XXX
6,Glaciers,ipccar5,glaciersfair,XXX
7,Glaciers,ipccar6,gmipemuglaciers,XXX
8,Glaciers,emulandice,glaciers,emuglaciers.emulandice.glaciers_globalsl.nc
9,GrIS,ipccar5,icesheets,XXX


#### $\color{Red}{\text{Use Workflow data from ==> facts-experiments/coupling.}}$

In [9]:
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];
YEAR        = 2100; # Percentile Year for the table.    
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for val in MODNO:
    for scenario in SCENARIO: 
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val];
        # Skip if data is absent.
        if df["Data_File"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        if scenario == 'ssp585':
            dataFOLDER = '/projects/kopp/facts-experiments/221115/coupling.{arg1}.localhost/output/'.format(arg1=scenario)
            dataFILE   = 'coupling.{arg1}.localhost.'.format(arg1=scenario) + df.loc[val].at['Data_File']
        else:
            dataFOLDER = '/projects/kopp/facts-experiments/221115/coupling.{arg1}/output/'.format(arg1=scenario)
            dataFILE   = 'coupling.{arg1}.'.format(arg1=scenario) + df.loc[val].at['Data_File']
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        if SUB_MODULE == 'temperature':
            GMSL    = (d_nc["surface_temperature"][:,Yind,0].values)/1   # Convert to meters.
        else:
            GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1      # Convert to meters.
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        TEMP = [COMP,MODULE,SUB_MODULE, scenario] + pcntle.tolist()
        df_TEMP.append( TEMP )
df_ptile = pd.DataFrame( df_TEMP, columns=['Component','Module','sub_Module','SSP', ] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
df_ptile[''] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', axis=1 )
df_ptile1    = pd.DataFrame( df_ptile.set_index( ['Component','Module','sub_Module','SSP'] )[''] ).unstack().swaplevel( 0,1, axis=1 )
df_ptile1

SSP                                                       ssp126  \
                                                                   
Component Module      sub_Module                                   
AIS       emulandice  AIS                 85.00 (32.83 , 146.17)   
          ipccar5     icesheets           65.50 (-8.00 , 140.00)   
          larmip      AIS                130.00 (56.00 , 259.00)   
GSAT      fair        temperature             1.64 (1.35 , 2.01)   
Glaciers  emulandice  glaciers            90.00 (68.83 , 110.00)   
          ipccar5     glaciers            90.00 (63.83 , 123.00)   
GrIS      FittedISMIP GrIS                 77.00 (61.00 , 97.17)   
          emulandice  GrIS                56.00 (16.00 , 104.00)   
LWS       ssp         landwaterstorage     33.00 (20.83 , 45.00)   
TE        tlm         oceandynamics     140.00 (114.00 , 173.00)   

SSP                                                       ssp245  \
                                                                   
Component Module      sub_Module                                   
AIS       emulandice  AIS                 82.00 (29.83 , 144.17)   
          ipccar5     icesheets          59.50 (-15.00 , 134.17)   
          larmip      AIS                137.00 (56.83 , 283.00)   
GSAT      fair        temperature             2.61 (2.20 , 3.14)   
Glaciers  emulandice  glaciers           122.00 (98.00 , 143.00)   
          ipccar5     glaciers           109.00 (79.00 , 147.00)   
GrIS      FittedISMIP GrIS               101.00 (83.00 , 124.00)   
          emulandice  GrIS                80.00 (40.00 , 126.00)   
LWS       ssp         landwaterstorage     33.00 (20.83 , 45.00)   
TE        tlm         oceandynamics     190.00 (157.00 , 231.17)   

SSP                                                       ssp585  
                                                                  
Component Module      sub_Module                                  
AIS       emulandice  AIS                 83.00 (31.66 , 144.00)  
          ipccar5     icesheets          40.00 (-37.00 , 117.00)  
          larmip      AIS                148.50 (59.83 , 338.34)  
GSAT      fair        temperature             4.68 (3.98 , 5.55)  
Glaciers  emulandice  glaciers          177.00 (150.00 , 202.00)  
          ipccar5     glaciers          159.00 (116.00 , 212.00)  
GrIS      FittedISMIP GrIS              139.00 (115.00 , 177.00)  
          emulandice  GrIS               126.00 (86.00 , 177.00)  
LWS       ssp         landwaterstorage     33.00 (20.83 , 45.00)  
TE        tlm         oceandynamics     293.00 (242.00 , 352.17)

In [4]:
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];
YEAR        = 2100; # Percentile Year for the table.    
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for scenario in SCENARIO:
    # Define pd DF columns to append.
    PERCENTILE  = '%ile({arg2})'.format(arg2=scenario)
    percentile = np.nan
    df[PERCENTILE] = percentile
    df[PERCENTILE] = df[PERCENTILE].astype('object') 
    #
    for val in MODNO: 
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val];
        # Skip if data is absent.
        if df["Data_File"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        if scenario == 'ssp585':
            dataFOLDER = '/projects/kopp/facts-experiments/coupling.{arg1}.localhost/output/'.format(arg1=scenario)
            dataFILE   = 'coupling.{arg1}.localhost.'.format(arg1=scenario) + df.loc[val].at['Data_File']
        else:
            dataFOLDER = '/projects/kopp/facts-experiments/coupling.{arg1}/output/'.format(arg1=scenario)
            dataFILE   = 'coupling.{arg1}.'.format(arg1=scenario) + df.loc[val].at['Data_File']
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        if SUB_MODULE == 'temperature':
            GMSL    = (d_nc["surface_temperature"][:,Yind,0].values)/1   # Convert to meters.
        else:
            GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1      # Convert to meters.
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        df.at[val,PERCENTILE] = [pcntle[0], pcntle[1],pcntle[2]]
        # Print the OP.
print('GMSL Component Projections for ',d_nc["years"][Yind].values)    
#show(df)
df

FileNotFoundError: [Errno 2] No such file or directory: b'/projects/kopp/facts-experiments/coupling.ssp126/output/coupling.ssp126.temperature.fair.temperature_gsat.nc'